In [ ]:
pip install names-dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 MB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.5 MB/s eta 0:00:00
  Created wheel for names-dataset: filename=names_dataset-3.1.0-py3-none-any.whl size=116832757 sha256=6eb107498e0da6c2347c3103e55332a5279169aeb4130a74c84f9ca82c49cd5e
  Stored in directory: /root/.cache/pip/wheels/cf/f8/43/0c4aba87b34e971e7255a41f11dc0035c5e55b026dc3480986
Successfully built names-dataset


In [ ]:
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text
import re
import spacy
from names_dataset import NameDataset
import string
# Creazione del database
engine = create_engine('sqlite:///Bechdel_test.db')

# Funzione per creare le tabelle
def create_tables():
    with engine.connect() as conn:
        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS MOVIE (
            film_id INTEGER PRIMARY KEY,
            titolo TEXT,
            anno INTEGER
        )
        '''))

        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS SCRIPT (
            film_id INTEGER,
            scene_id INTEGER,
            place TEXT,
            scena TEXT,
            personaggi TEXT,
            count_female INTEGER,
            count_male INTEGER,
            nom TEXT,
            count_male_nom INTEGER,
            PRIMARY KEY (film_id, scene_id)
        )
        '''))

        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS CAST (
            film_id INTEGER,
            nome TEXT,
            ruolo TEXT,
            gender TEXT,
            PRIMARY KEY (film_id, nome)
        )
        '''))

create_tables()

# Funzione per dividere lo script mantenendo INT ed EXT
def dividere_script_con_int_ext(script):
    # Carica il modello spaCy
    nlp = spacy.load('en_core_web_sm')

    # Trova le posizioni in cui appare INT o EXT
    int_ext_positions = [match.start() for match in re.finditer(r'\bINT\b|\bEXT\b', script)]
    # Dividi il testo dello script in base alle posizioni trovate
    scene_divise = []
    for start, end in zip([0] + int_ext_positions, int_ext_positions + [len(script)]):
        scena = script[start:end].strip()
        if scena:
            place = scena.split('\n', 1)[0].strip() if '\n' in scena else scena.strip()

            # Estrai solo INT o EXT dalla stringa di place
            match = re.match(r'\b(INT|EXT)\b', place)
            if match:
                place = match.group(0)

            # Controlla se la scena inizia con INT./, EXT./, INT/, o EXT/
            if re.match(r'\b(INT\./|EXT\./|INT/|EXT/)', scena):
                continue  # Salta questa scena
            scena_pulita = re.sub(r'[<>*\d]', '', scena)
            # Estrai i personaggi dalla scena utilizzando spaCy
            doc = nlp(scena)
            personaggi = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
            scene_divise.append({'place': place, 'scena': scena_pulita, 'personaggi': ' '.join(personaggi)})
    return scene_divise

# Funzione per scaricare lo script da IMSDb e dividerlo in scene
def scarica_e_dividi_script_da_imsdb(titolo_film):
    url = f"https://www.imsdb.com/scripts/{titolo_film.replace(' ', '-')}.html"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        script = soup.find('td', {'class': 'scrtext'}).text
        scene_divise = dividere_script_con_int_ext(script)
        return scene_divise
    else:
        print(f"Errore nel scaricare lo script da IMSDb per il film {titolo_film}")
        return None

# Funzione per scaricare il cast da IMDb

def scarica_cast_da_imdb(imdb_id):
    url = f"https://www.imdb.com/title/{imdb_id}/fullcredits"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        cast_list = []
        roles_seen = set()

        for row in soup.select('.cast_list tr')[1:]:  # Ignore the header row
            columns = row.find_all('td')
            if len(columns) >= 4:
                actor_name = columns[1].get_text(strip=True)
                character_name = columns[3].get_text(strip=True).split(' ')[0]  # Take only the first word of the role

                # Remove all punctuation from the character name
                character_name = character_name.translate(str.maketrans('', '', string.punctuation))

                # Remove trailing 's if present
                character_name = re.sub(r"'s$", "", character_name)

                # Check if the character name has already been seen
                if character_name not in roles_seen:
                    roles_seen.add(character_name)

                    # Exclude roles that are "A", "The", contain numbers, or contain "uncredited"
                    if (character_name.lower() in {"a", "the"} or any(char.isdigit() for char in character_name)
                            or "uncredited" in character_name.lower()):
                        continue  # Skip this row

                    cast_list.append({'name': actor_name, 'role': character_name})

        return cast_list
    else:
        print(f"Error: Unable to fetch data from IMDB for {imdb_id}")
        return []



# Funzione per inserire i dati nel database
def insert_data(data, table_name):
    with engine.begin() as conn:
        if table_name == "SCRIPT":
            conn.execute(text('''
            INSERT INTO SCRIPT (film_id, scene_id, place, scena, personaggi, count_female, count_male, nom, count_male_nom)
            VALUES (:film_id, :scene_id, :place, :scena, :personaggi, :count_female, :count_male, :nom, :count_male_nom)
            '''), data)
        elif table_name == "CAST":
            # Check if the entry already exists before inserting
            existing_entry = conn.execute(text('''
                SELECT 1
                FROM CAST
                WHERE film_id = :film_id AND nome = :nome
            '''), data).fetchone()
            if not existing_entry:  # Insert only if it doesn't exist
                conn.execute(text('''
                INSERT INTO CAST (film_id, nome, ruolo, gender)
                VALUES (:film_id, :nome, :ruolo, :gender)
                '''), data)
        elif table_name == "MOVIE":
            conn.execute(text('''
            INSERT INTO MOVIE (film_id, titolo, anno)
            VALUES (:film_id, :titolo, :anno)
            '''), data)

def pulisci_ruolo(ruolo):
    return re.sub(r'\([^)]*\)', '', ruolo).strip()

# Funzione per pulire la scena
def pulisci_scena(scena):
    return scena.split('\n', 1)[1].strip() if '\n' in scena else scena.strip()

# Funzione per determinare il genere di un nome
def gender_name(name):
    # Inizializza il dataset dei nomi
    nd = NameDataset()
    # Cerca il nome nel dataset
    name_info = nd.search(name)
    if name_info and 'first_name' in name_info and name_info['first_name']:
        # Ottieni le informazioni sul genere
        gender_info = name_info['first_name'].get('gender')
        if gender_info:
            # Estrai il genere più probabile
            most_probable_gender = max(gender_info, key=gender_info.get)
            return most_probable_gender
    # Se non ci sono informazioni sul genere, restituisci 'unknown'
    return None

# Funzione per estrarre i personaggi comuni tra scena e ruoli
def estrai_personaggi_comuni(scena, ruoli, parole_chiave, ):
    # Dizionario per mappare le parole pulite alle loro versioni originali
    parole_originali = {}

    # Rimuove la punteggiatura da ogni parola nella scena e costruisce il dizionario
    personaggi_scena = []
    for word in scena.split():
        parola_pulita = word.strip(string.punctuation)
        if parola_pulita.isupper():  # Controlla se la parola è interamente maiuscola
            personaggi_scena.append(parola_pulita.lower())  # Aggiunge la parola in minuscolo alla lista
            parole_originali[parola_pulita.lower()] = parola_pulita  # Mappa la parola in minuscolo alla sua forma originale

    # Converte i ruoli e le parole chiave in minuscolo
    ruoli_set = set(ruolo.lower() for ruolo in ruoli)
    parole_chiave_set = set(parola.lower() for parola in parole_chiave)

    # Trova gli elementi comuni tra i tre insiemi
    personaggi_comuni_set = set()
    for personaggio in personaggi_scena:
        if personaggio in ruoli_set.union(parole_chiave_set):
            personaggi_comuni_set.add(parole_originali[personaggio])

    # Restituisce le parole nel loro formato originale, mantenendo l'ordine originale
    personaggi_comuni = [personaggio for personaggio in parole_originali.values() if personaggio in personaggi_comuni_set]

    return ' '.join(personaggi_comuni)


def estrai_ruoli_comuni(scena, ruoli_cast, parole_chiave_maschi):
    # Rimuove la punteggiatura dalla scena
    scena_pulita = re.sub(r'[{}]'.format(re.escape(string.punctuation)), '', scena)

    # Trova le parole con iniziale maiuscola e le abbreviazioni
    parole_iniziali_maiuscole = re.findall(r'\b[A-Z][a-z]*\.?\b', scena_pulita)

    # Converte i ruoli_cast e le parole chiave in un set per la corrispondenza più efficiente
    ruoli_cast_set = set(ruolo.lower() for ruolo in ruoli_cast)
    parole_chiave_set = set(parola.lower() for parola in parole_chiave_maschi)

    # Trova le parole comuni tra le parole iniziali maiuscole e i ruoli_cast
    ruoli_comuni_set = set()
    for parola in parole_iniziali_maiuscole:
        parola_minuscola = parola.lower()
        if parola_minuscola in ruoli_cast_set:
            ruoli_comuni_set.add(parola)

    # Trova le parole chiave direttamente nella scena
    parole_chiave_trovate = set()
    parole_nella_scena = scena_pulita.lower().split()
    for parola in parole_nella_scena:
        if parola in parole_chiave_set:
            parole_chiave_trovate.add(parola)

    # Restituisce i ruoli comuni e le parole chiave trovate come stringa separata da spazi
    return ' '.join(ruoli_comuni_set) + ' ' + ' '.join(parole_chiave_trovate)



# Funzione per aggiornare la colonna count_female nella tabella SCRIPT


def update_script_counts(parole_chiave, parole_chiave_maschi, film_id):

    with engine.begin() as conn:
        # Seleziona i ruoli di tutti i personaggi dalla tabella CAST per il film specificato
        cast_roles = conn.execute(text('''
            SELECT ruolo, LOWER(gender)
            FROM CAST
            WHERE film_id = :film_id
        '''), {'film_id': film_id}).fetchall()

        cast_roles_map = {'male': set(), 'female': set()}
        for role, gender in cast_roles:
            if gender == 'male':
                cast_roles_map['male'].add(role.lower())
            elif gender == 'female':
                cast_roles_map['female'].add(role.lower())

        # Converti le parole chiave in minuscolo
        parole_chiave_set = set(parola.lower() for parola in parole_chiave)
        parole_chiave_maschi_set = set(parola.lower() for parola in parole_chiave_maschi)

        # Rimuovi le parole chiave dalle liste dei ruoli del cast
        cast_roles_map['male'] -= parole_chiave_set
        cast_roles_map['male'] -= parole_chiave_maschi_set
        cast_roles_map['female'] -= parole_chiave_set
        cast_roles_map['female'] -= parole_chiave_maschi_set

        scripts = conn.execute(text('''
            SELECT film_id, scene_id, personaggi
            FROM SCRIPT
            WHERE film_id = :film_id
        '''), {'film_id': film_id}).fetchall()

        # Calcola il conteggio degli elementi comuni e aggiorna la tabella SCRIPT
        for script in scripts:
            personaggi = script[2].lower().split()
            common_count = sum(1 for personaggio in personaggi if personaggio in cast_roles_map['female'])
            keyword_count = sum(1 for personaggio in personaggi if personaggio in parole_chiave_set)
            male_count = sum(1 for personaggio in personaggi if personaggio in cast_roles_map['male'])
            keyword_male_count = sum(1 for personaggio in personaggi if personaggio in parole_chiave_maschi_set)
            total_count = common_count + keyword_count
            total_male_count = male_count + keyword_male_count
            conn.execute(text('''
                UPDATE SCRIPT
                SET count_female = :count_female,
                    count_male = :count_male
                WHERE film_id = :film_id AND scene_id = :scene_id
            '''), {'count_female': total_count, 'count_male': total_male_count, 'film_id': script[0], 'scene_id': script[1]})




def update_script_male_nom_counts(parole_chiave_maschi):

    parole_chiave_maschi_set = set(parola.lower() for parola in parole_chiave_maschi)

    with engine.begin() as conn:
        # Seleziona le righe dalla tabella SCRIPT
        scripts = conn.execute(text('''
            SELECT film_id, scene_id, nom
            FROM SCRIPT
        ''')).fetchall()

        for script in scripts:
            count_male_nom = 0
            if script[2]:  # Verifica se la colonna nom non è vuota
                nom_names = script[2].split()  # Dividi la stringa in parole
                for name in nom_names:
                    # Verifica se il nome è presente tra le parole chiave maschili
                    if name.lower() in parole_chiave_maschi_set:
                        count_male_nom += 1
                    else:
                        # Se non è tra le parole chiave, cerca nella tabella CAST per ruoli maschili
                        result = conn.execute(text('''
                            SELECT 1
                            FROM CAST
                            WHERE film_id = :film_id
                                AND LOWER(gender) = 'male'
                                AND LOWER(ruolo) = :ruolo
                        '''), {'film_id': script[0], 'ruolo': name.lower()}).fetchone()
                        if result:
                            count_male_nom += 1

            # Aggiorna la colonna count_male_nom nella riga corrente
            conn.execute(text('''
                UPDATE SCRIPT
                SET count_male_nom = :count_male_nom
                WHERE film_id = :film_id AND scene_id = :scene_id
            '''), {'count_male_nom': count_male_nom, 'film_id': script[0], 'scene_id': script[1]})

# Funzione principale
def main():
    # Lista di film con titoli e ID IMDb
    films = [
        {"titolo": "Rocky", "imdb_id": "tt0075148", "anno": 1976},
        {"titolo": "Blade Runner","imdb_id": "tt0083658", "anno":1982 },
        {"titolo": "Dead Poets Society", "imdb_id": "tt0097165", "anno": 1989},
        {"titolo": "Fargo", "imdb_id": "tt0116282", "anno": 1996},
        {"titolo": "Good Will Hunting", "imdb_id": "tt0119217", "anno": 1997},
        {"titolo": "Truman Show, The", "imdb_id": "tt0120382", "anno": 1998},
        {"titolo": "American Beauty", "imdb_id": "tt0169547", "anno": 1999},
        {"titolo": "Gladiator", "imdb_id": "tt0172495", "anno": 2000},
        {"titolo": "Memento", "imdb_id": "tt0209144", "anno": 2000},
        {"titolo": "American Psycho", "imdb_id": "tt0144084", "anno": 2000},
        {"titolo": "Big Fish", "imdb_id": "tt0319061", "anno": 2003},
        {"titolo": "Eternal Sunshine of the Spotless Mind", "imdb_id": "tt0338013", "anno": 2004},
        {"titolo": "Devil Wears Prada, The", "imdb_id": "tt0458352", "anno": 2006},
        {"titolo": "No Country for Old Men", "imdb_id": "tt0477348", "anno": 2007},
        {"titolo": "Anna Karenina", "imdb_id": "tt1781769", "anno": 2012},
        {"titolo": "12 Years a Slave", "imdb_id": "tt2024544", "anno": 2013},
        {"titolo": "Great Gatsby, The", "imdb_id": "tt1343092", "anno": 2013},
        {"titolo": "Interstellar", "imdb_id": "tt0816692", "anno": 2014},
        {"titolo": "American Sniper", "imdb_id": "tt2179136", "anno": 2014},
        {"titolo": "La La Land", "imdb_id": "tt3783958", "anno": 2016},
    ]

    parole_chiave = [
        "LADY", "WOMAN", "MOM", "Mother","AUNT", "WIFE", "GIRLFRIEND", "MOTHER","GRANDMOTHER", "GRANDMA", "FEMALE", "GIRL", "ACTRESS", "MISS", "MS.", "MRS","ASSISTANT", "NURSE","princess","Queen"
    ]
    parole_chiave_maschi = [
        "Sir", "Man", "Dad", "Uncle", "Husband", "Boy", "Father","Grandad", "Grandfather", "Male","Prince", "Mr", "King"
    ]

    for film_id, film in enumerate(films, start=1):
        titolo_film = film["titolo"]
        imdb_id = film["imdb_id"]
        anno = film["anno"]

        # Scarica il cast da IMDb
        cast_list = scarica_cast_da_imdb(imdb_id)

        # Inserimento dati nella tabella CAST se non esistono già nelle tabelle FEMALE e MALE
        with engine.begin() as conn:
            for actor in cast_list:
                first_name = actor['name'].split(' ', 1)[0]
                most_probable_gender = gender_name(first_name)
                if most_probable_gender is None:
                # Se il genere è sconosciuto, passa alla prossima iterazione del ciclo
                    continue
                print(f"Determinato gender per {first_name} ({actor['name']}): {most_probable_gender}")
                cast_data = {
                    'film_id': film_id,
                    'nome': actor['name'],
                    'ruolo': pulisci_ruolo(actor['role']),
                    'gender': most_probable_gender,
                }
                insert_data(cast_data, 'CAST')

        # Scarica e divide lo script da IMSDb
        scene_divise = scarica_e_dividi_script_da_imsdb(titolo_film)

        # Estrai i ruoli da CAST per i personaggi delle scene
        ruoli_cast = [pulisci_ruolo(actor['role']) for actor in cast_list]

        if scene_divise:
            # Svuota la lista dei ruoli del cast e riempila con i nuovi ruoli
            ruoli_cast.clear()
            ruoli_cast.extend(pulisci_ruolo(actor['role']) for actor in cast_list)
            print(ruoli_cast)
            # Inserisci le scene nel database e calcola count_male, count_female e count_nom
            for scene_id, scene in enumerate(scene_divise, start=1):
                place = scene['place']
                cleaned_scene = pulisci_scena(scene['scena'])
                ruoli_comuni = estrai_ruoli_comuni(scene['scena'], ruoli_cast, parole_chiave_maschi)
                personaggi_comuni = estrai_personaggi_comuni(scene['scena'], ruoli_cast, parole_chiave)

                # Calcola count_female e count_male_nom
                count_female = sum(1 for personaggio in personaggi_comuni.split() if personaggio in parole_chiave)

                count_male_nom = sum(1 for nome in ruoli_comuni.split() if nome in parole_chiave_maschi)

                scene_data = {
                    'film_id': film_id,
                    'scene_id': scene_id,
                    'place': place,
                    'scena': cleaned_scene,
                    'personaggi': personaggi_comuni,
                    'count_female': count_female,
                    'count_male': 0,  # Calcolato in update_script_counts
                    'nom': ruoli_comuni,
                    'count_male_nom': count_male_nom,
                }
                insert_data(scene_data, 'SCRIPT')

            print(f"Scene del film {titolo_film} inserite nel database.")

        # Inserimento dati nella tabella MOVIE
        movie_data = {'film_id': film_id, 'titolo': titolo_film, 'anno': anno}
        insert_data(movie_data, 'MOVIE')

        # Aggiorna la colonna count_female nella tabella SCRIPT
        update_script_counts(parole_chiave, parole_chiave_maschi, film_id)
        update_script_male_nom_counts(parole_chiave_maschi)

if __name__ == "__main__":
    main()


Determinato gender per Sylvester (Sylvester Stallone): Male
Determinato gender per Talia (Talia Shire): Female
Determinato gender per Burt (Burt Young): Male
Determinato gender per Carl (Carl Weathers): Male
Determinato gender per Burgess (Burgess Meredith): Male
Determinato gender per Thayer (Thayer David): Male
Determinato gender per Joe (Joe Spinell): Male
Determinato gender per Jimmy (Jimmy Gambina): Male
Determinato gender per Bill (Bill Baldwin): Male
Determinato gender per Al (Al Silvani): Male
Determinato gender per George (George Memmoli): Male
Determinato gender per Jodi (Jodi Letizia): Female
Determinato gender per Diana (Diana Lewis): Female
Determinato gender per Stan (Stan Shaw): Male
Determinato gender per Don (Don Sherman): Male
Determinato gender per Billy (Billy Sands): Male
Determinato gender per Shirley (Shirley O'Hara): Female
Determinato gender per Kathleen (Kathleen Parker): Female
Determinato gender per Frank (Frank Stallone): Male
Determinato gender per Lloyd (